In [105]:
from spacy.lang.en import English
from textacy.preprocess import preprocess_text

nlp = English()

def preprocess(ref):
    
    doc = nlp(ref)
    ref = ' '.join((token.text for token in doc))
    
    return preprocess_text(ref, no_accents=True, lowercase=True, no_punct=True)

In [106]:
SUBMISSION_FILEPATHS = [
    "../data/webnlg2017/submissions/melbourne/final_result.txt",
    "../data/webnlg2017/submissions/tilburg/smt_test.out.ordered",
    "../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt",
    "../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt",
    "../data/webnlg2017/submissions/tilburg/template_test.out.ordered",
    "../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered",
    "../data/webnlg2017/submissions/baseline_sorted.txt",
    "../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt",
    "../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt"
]

# BLEU

## Preprocessing submited files

In [116]:
for filepath in SUBMISSION_FILEPATHS:
    
    with open(filepath) as f, open(filepath + '_ascii', 'w') as f_out:

        for ref in f:

            f_out.write(preprocess(ref) + '\n')

## Let's have a look at reference texts

In [117]:
import xml.etree.ElementTree as ET

tree = ET.parse("../data/webnlg2017/testdata_with_lex.xml")
root = tree.getroot()

references = []

for entry in root.iter('entry'):
    
    references_of_entry = []
    for ref in entry.findall('lex'):
        
        ref_final = preprocess(ref.text)
        
        references_of_entry.append(ref_final)
    
    references.append(references_of_entry)

In [118]:
references[0]

['abilene texas is served by the abilene regional airport',
 'abilene regional airport serves the city of abilene in texas']

## Let's calculate BLEU with NLTK

In [119]:
from nltk.translate.bleu_score import corpus_bleu

with open('../data/webnlg2017/submissions/melbourne/final_result.txt_ascii') as f:
    
    candidates = [line for line in f.readlines()]

split_candidates = [candidate.split() for candidate in candidates]
split_references = [[reference_text.split() for reference_text in reference] for reference in references]

corpus_bleu(split_references, split_candidates)

0.4304633737903923

In [120]:
from nltk.translate.bleu_score import sentence_bleu

In [121]:
sentence_bleu(split_references[0], split_candidates[0])

0.9193227152249185

# Let's calculate BLEU with multi-bleu.perl

https://github.com/moses-smt/mosesdecoder/blob/master/scripts/generic/multi-bleu.perl

## First we need to generate reference files

In [136]:
import xml.etree.ElementTree as ET
from spacy.lang.en import English

nlp = English()

tree = ET.parse('test_reference_files_bleu/testdata_with_lex.xml')
root = tree.getroot()

references = []
for entry in root.iter('entry'):
    
    references_of_entry = []
    for ref in entry.findall('lex'):
        
        ref_final = preprocess(ref.text)
        
        references_of_entry.append(ref_final)
    
    references.append(references_of_entry)

In [137]:
ref_per_id = [[] for _ in range(8)]

for i in range(8):
    
    for reference_list in references:
        
        if len(reference_list) > i:
            
            ref_per_id[i].append(reference_list[i])
        else:
            ref_per_id[i].append('')

In [138]:
for i, refs in enumerate(ref_per_id):
    
    with open('test_reference_files_bleu/ref_{}.txt'.format(i), 'w') as f:
        
        f.writelines(('{}\n'.format(ref) for ref in refs))

In [139]:
!head -n 20 ../data/webnlg2017/submissions/melbourne/final_result.txt_ascii

abilene regional airport serves the city of abilene texas
adolfo suarez madrid barajas airport is located in madrid paracuellos de jarama san sebastian de los reyes and alcobendas
18l 36r is the runway name of adolfo suarez madrid barajas airport new zealand
the icao location identifier of afonso pena international airport is sbct
afonso pena international airport serves the city of curitiba
al taqaddum air base serves the city of fallujah
the runway length of al taqaddum air base is 3684 0
the runway name of alderney airport is 14 32
the runway length of allama iqbal international airport is 3360 12
the number of the first runway at amsterdam airport schiphol is 18
the 1st runway at amsterdam airport schiphol is made from asphalt
the runway name of amsterdam airport schiphol is 06 24 kaagbaan
andrews county airport is 973 0 metres above sea level
andrews county texas is the owner of andrews county airport
the runway length of andrews county airport is 896 0
the 1st runway at angola in

In [140]:
!head -n 20 test_reference_files_bleu/ref_0.txt

abilene texas is served by the abilene regional airport
adolfo suarez madrid barajas airport can be found in madrid paracuellos de jarama san sebastian de los reyes and alcobendas
the runway name of adolfo suarez madrid barajas airport is 18l 36r
afonso pena international airport icao location idenitifier is sbct
afonso pena international airport serves the city of curitiba
the al taqaddum air base serves the city of fallujah
the runway length of al taqaddum air base is 3684 0
alderney airport runway name is 14 32
the runway length at allama iqbal international airport is 3 360 12
the first runway at amsterdam s schiphol airport is known as number 18
the 5th runway at amsterdam airport schiphol has an asphalt surfacing
amsterdam airport schiphol runway name is 06 24 kaagbaan
andrews county airport is 973 metres above sea level
the andrews county airport is owned by andrews county texas
the runway length of andrews county airport is 896
angola international airport is the first runway m

## And then let's try multi-bleu.perl

## Melbourne

In [141]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/melbourne/final_result.txt_ascii

BLEU = 43.11, 79.6/55.0/36.8/25.3 (BP=0.960, ratio=0.961, hyp_len=34390, ref_len=35799)


## Tilb-SMT

In [142]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/tilburg/smt_test.out.ordered_ascii

BLEU = 44.34, 76.0/54.3/37.3/26.1 (BP=0.991, ratio=0.991, hyp_len=36180, ref_len=36518)


## PKUWriter

In [129]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/pkuwriter/PKUWriter_results.txt_ascii

BLEU = 37.93, 73.6/50.2/34.5/24.2 (BP=0.906, ratio=0.910, hyp_len=31950, ref_len=35119)


## UPF-FORGe

In [130]:
    !../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/upf/UPF_All_sent_final.txt_ascii

BLEU = 39.07, 77.6/50.7/31.2/19.0 (BP=1.000, ratio=1.012, hyp_len=37497, ref_len=37038)


## Tilb-Pipeline

In [131]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/tilburg/template_test.out.ordered_ascii

BLEU = 33.71, 82.1/56.9/37.6/24.9 (BP=0.737, ratio=0.766, hyp_len=25481, ref_len=33257)


## Tilb-NMT

In [132]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/tilburg/nmt_test.out.ordered_ascii

BLEU = 33.00, 68.4/42.4/25.8/15.8 (BP=1.000, ratio=1.036, hyp_len=38328, ref_len=37011)


## Baseline

In [133]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/baseline_sorted.txt_ascii

BLEU = 32.61, 62.8/41.9/30.9/23.8 (BP=0.874, ratio=0.881, hyp_len=30830, ref_len=34975)


## Adapt

In [134]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/adaptCenter/ADAPTcentreWebNLGsubmission.txt_ascii

BLEU = 30.31, 53.2/35.3/24.8/18.2 (BP=1.000, ratio=1.333, hyp_len=51161, ref_len=38389)


## UIT-VNU

In [135]:
!../evaluation/webnlg2017/webnlg-baseline-master/multi-bleu.perl test_reference_files_bleu/ref_0.txt test_reference_files_bleu/ref_1.txt test_reference_files_bleu/ref_2.txt < ../data/webnlg2017/submissions/uit-danglt-clnlp/Submission-UIT-DANGNT-CLNLP.txt_ascii

BLEU = 7.09, 65.1/53.1/37.3/26.7 (BP=0.165, ratio=0.357, hyp_len=9793, ref_len=27458)
